<a href="https://colab.research.google.com/github/LomaxOS/AI-Assignments/blob/main/LomaxOS_SearchTreeNode_Hare%26HoundsGame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hare and hounds game
from collections import defaultdict

class CurrentBoard:
    def __init__(self, board = None):

        # initiate with the hounds at 0, 1, 3 and hare at 10
        self.board = board if board is not None else ['Dog', 'Dog', ' ', 'Dog', ' ', ' ', ' ', ' ', ' ', ' ', 'Hare']

        #Allowed moves of dogs
        self.allowed_moves_for_dog = defaultdict(list, {
            0: [1, 2, 3],
            1: [2, 4, 5],
            2: [1, 3, 5],
            3: [2, 4, 5],
            4: [5, 9],
            5: [4, 6, 7, 8, 9],
            6: [5, 7],
            7: [8, 10],
            8: [7, 9, 10],
            9: [8, 10]
        })
        #the adjancency list of all possible moves for each location on the board
        self.moves_at_location_of = defaultdict(list, {
            0: [1, 2, 3],
            1: [0, 2, 5, 6],
            2: [0, 1, 3, 5],
            3: [0, 2, 5, 4],
            4: [3, 5, 9],
            5: [1, 2, 3, 4, 6, 7, 8, 9],
            6: [1, 5, 7],
            7: [6, 5, 8, 10],
            8: [5, 7, 9, 10],
            9: [5, 4, 8, 10],
            10: [7, 8, 9]
        })

        self.state = self.state_of_board()


    # Display the board
    def display(self):
        pos = self.board

        print("       1({})------ 6({})------ 7({})".format(pos[1], pos[6], pos[7]))
        print("   /       |    \     |    /     |    \\")
        print("0({})---2({})------ 5({})------ 8({})---- 10({})".format(pos[0], pos[2], pos[5], pos[8], pos[10]))
        print("   \       |    /     |    \     |    /")
        print("       3({})------ 4({})------ 9({})".format(pos[3], pos[4], pos[9]))



    #switch from current player
    def other(self,piece):
      if piece == "Dog":
        return "Hare"
      return "Dog"
 #get all possible moves at the index location - for instance: index 2 will have [0, 1, 3, 5]
    #Get all possible moves
    def all_possible_moves(self, player_piece):

        list_of_possible_moves = []

        for index, spot in enumerate(self.board):

            if spot == player_piece:

                adjacent_moves_location = self.moves_at_location_of[index]#Get

                for aLoc in adjacent_moves_location:

                    if self.is_an_empty_space_at(aLoc):

                        if player_piece == 'Dog' and self.is_not_a_backward_move(index, aLoc): #Ensure no backward moves

                          list_of_possible_moves.append(CurrentBoard(self.get_description_for(player_piece, index, aLoc)))

        return list_of_possible_moves


    #return true if a location adjacent to this index is empty
    def is_an_empty_space_at(self, loc):
        return self.board[loc] == ' '

    #check Dogs allowed moves
    #return true if new_location is on the list of allowed moves from current_location
    def is_not_a_backward_move(self, current_location, new_location):

        return new_location in self.allowed_moves_for_dog[current_location]

    #return new board
    def get_description_for(self, piece, current_location, new_location):

      new_board = self.board[:]
      new_board[new_location] = piece
      new_board[current_location] = ' '

      return new_board

    #winning state based on the hare location
    def state_of_board(self):

      hare_location = self.board.index('Hare')
      hare_moves = self.moves_at_location_of[hare_location]

      # Check if the hare is trapped
      is_hare_trapped = True

      for loc in hare_moves:
          if self.board[loc] == ' ':  #Check if there is at least one free space around the hare
              is_hare_trapped = False
              break

      if is_hare_trapped:
          return "Dogs"  #dogs win

      # winning conditions for the Hare
      hare_winning_locations = {
          1: [0],
          2: [0],
          3: [0],
          4: [0, 2, 3],
          5: [1, 2],
          6: [0, 1, 2]
      }

      # check if the Hare is in any of its winning locations
      if hare_location in hare_winning_locations:

          if all(self.board[loc] == ' ' for loc in hare_winning_locations[hare_location]):
              return "Hare"

      return "Continue"



In [ ]:
cd = CurrentBoard()

In [ ]:
cd.display()

       1(Dog)------ 6( )------ 7( )
   /       |    \     |    /     |    \
0(Dog)---2( )------ 5( )------ 8( )---- 10(Hare)
   \       |    /     |    \     |    /
       3(Dog)------ 4( )------ 9( )


In [ ]:
class SearchTreeNode:
    def __init__(self, board_instance, playing_as, ply=0):
        self.children = []
        self.value_is_assigned = False
        self.ply_depth = ply
        self.current_board = board_instance
        self.move_for = playing_as
        self.value = 0

        game_state = self.current_board.state

        if game_state == "Continue":

            self.generate_children()
        else:
            #Assigning values based on the game state
            if game_state == "Dogs":

              if playing_as == "Dog":
                self.value = 1
              else :
                self.value = -1

            elif game_state == "Hare":

              if playing_as == "Hare":
                self.value = 1
              else:
                self.value = -1
            self.value_is_assigned = True

    def min_max_value(self):
        if self.value_is_assigned:
            return self.value

        if (self.ply_depth % 2) == 0:  #Max player's move
            self.value = float('-inf')
            for child in self.children:
                child_value = child.min_max_value()
                self.value = max(self.value, child_value)
        else:  #Min player's move
            self.value = float('inf')
            for child in self.children:
                child_value = child.min_max_value()
                self.value = min(self.value, child_value)

        self.value_is_assigned = True
        return self.value

    def generate_children(self):
        for board_for_next_move in self.current_board.all_possible_moves(self.move_for):
            self.children.append(SearchTreeNode(board_for_next_move, self.current_board.other(self.move_for), ply=self.ply_depth + 1))

In [ ]:
def play_Hare_and_Hounds():

  response = input("Do you wish to play first (y/n) ?")
  players_turn  = (response == "y")

  response = input("Do you wish to play Hare or Dogs  (Hare/Dog) ?")
  cb = CurrentBoard()
  player_is_playing = cb.other(cb.other(response))


  while cb.state_of_board() == "Continue":
    if players_turn:
      cb.display()
      print("Current positions:", cb.board)
      current_location = int(input("Move from: "))
      new_location = int(input("Move to: "))

      cb = CurrentBoard(cb.get_description_for(player_is_playing, current_location, new_location))
    else:
      search_tree = SearchTreeNode(cb,cb.other(player_is_playing))
      search_tree.min_max_value()
      cb = search_tree.children[-1].current_board

    if cb.state_of_board() != "Continue":
        cb.display()
        if cb.state_of_board() == "Hare":
          print("Hare wins!")
        else:
          print("Dogs wins!")

        break

    players_turn = not players_turn

if __name__ == "__main__":
    play_Hare_and_Hounds()

Do you wish to play first (y/n) ?n
Do you wish to play Hare or Dogs  (Hare/Dog) ?Hare
       1(Dog)------ 6( )------ 7( )
   /       |    \     |    /     |    \
0(Dog)---2( )------ 5( )------ 8( )---- 10(Hare)
   \       |    /     |    \     |    /
       3( )------ 4(Dog)------ 9( )
Current positions: ['Dog', 'Dog', ' ', ' ', 'Dog', ' ', ' ', ' ', ' ', ' ', 'Hare']
Move from: 10
Move to: 8
       1(Dog)------ 6( )------ 7( )
   /       |    \     |    /     |    \
0(Dog)---2( )------ 5( )------ 8(Hare)---- 10( )
   \       |    /     |    \     |    /
       3( )------ 4( )------ 9(Dog)
Current positions: ['Dog', 'Dog', ' ', ' ', ' ', ' ', ' ', ' ', 'Hare', 'Dog', ' ']
Move from: 8
Move to: 5
       1(Dog)------ 6( )------ 7( )
   /       |    \     |    /     |    \
0(Dog)---2( )------ 5(Hare)------ 8( )---- 10(Dog)
   \       |    /     |    \     |    /
       3( )------ 4( )------ 9( )
Current positions: ['Dog', 'Dog', ' ', ' ', ' ', 'Hare', ' ', ' ', ' ', ' ', 'Dog']
Move from:

KeyboardInterrupt: Interrupted by user